In [7]:
import pyspark
try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

# 1) (***) Tenemos información sobre
recetas en 3 RDD de Spark.
Recetas: (id_receta, nombre,
tiempo_preparación, dificultad)
Ingredientes: (id_ingrediente, nombre)
Ingredientes por Receta: (id_receta,
id_ingrediente, cantidad)
Se pide:
a) Obtener el nombre de todas las
recetas que tengan Cordero. (7 puntos)
b) Calcular la cantidad total de cada
ingrediente si queremos hacer todas las
recetas con Cordero que sean fáciles. (8
puntos)

In [2]:
recetas = [
    (1, "super hamburguer", 1, "facil"),
    (2, "ravioles con pesto", 2, "mediana"),
    (3, "cordero patagonico", 3, "facil"),
    (4, "hamburguer con papas", 1, "facil"),
   (5, "salmon rosado", 3, "dificil"),
    (6, "asado criollo", 4, "dificil")
]
ingredientes = [
    (1, "hamburguesa"),
    (2, "ravioles"),
    (3, "cordero"),
    (4, "papa"),
   (5, "salmon"),
    (6, "asado"),
        (7, "aceite"),
    (8, "sal"),
    (9, "salsa roja"),
    (10, "queso"),
   (11, "tomate"),
    (12, "lechuga")
]
ingredientes_por_Receta= [
    (1,3,1),
    (1,10,3),
    (1,11,1),
    (1,12,1),
    (2,2,1),
    (2,8,5),
    (2,9,1),
    (2,10,1),
    (3,3,2),
    (3,4,1),
    (3,11,1),
    (4,10,1),
    (4,11,1),
    (4,12,1),
      (5,5,2),
    (5,4,1),
    (6,6,1),
    (6,8,2),
      (6,4,3),
    
]

In [3]:
ingredientes

[(1, 'hamburguesa'),
 (2, 'ravioles'),
 (3, 'cordero'),
 (4, 'papa'),
 (5, 'salmon'),
 (6, 'asado'),
 (7, 'aceite'),
 (8, 'sal'),
 (9, 'salsa roja'),
 (10, 'queso'),
 (11, 'tomate'),
 (12, 'lechuga')]

In [5]:
ingredientes_por_Receta

[(1, 3, 1),
 (1, 10, 3),
 (1, 11, 1),
 (1, 12, 1),
 (2, 2, 1),
 (2, 8, 5),
 (2, 9, 1),
 (2, 10, 1),
 (3, 3, 2),
 (3, 4, 1),
 (3, 11, 1),
 (4, 10, 1),
 (4, 11, 1),
 (4, 12, 1),
 (5, 5, 2),
 (5, 4, 1),
 (6, 6, 1),
 (6, 8, 2),
 (6, 4, 3)]

In [24]:
ingredientesRDD=sc.parallelize(ingredientes)
recetasRDD=sc.parallelize(recetas)
ingredientes_por_RecetaRDD=sc.parallelize(ingredientes_por_Receta)

# Todas las recetas que  tengan cordero

In [36]:
id_cordero=ingredientesRDD.filter(lambda x:x[1]=="cordero").collect()
id_cordero[0][0]

3

In [58]:
recetasCordero=ingredientes_por_RecetaRDD.filter(lambda x:x[1]==id_cordero[0][0])\
                                .map(lambda x: (x[0],(x[1],x[2])))\
                            .join(recetasRDD.map(lambda x:(x[0],(x[1],x[3]))))
recetasCordero.collect()

[(1, ((3, 1), ('super hamburguer', 'facil'))),
 (3, ((3, 2), ('cordero patagonico', 'facil')))]

In [59]:
recetasCordero.map(lambda x:x[1][1][0]).collect()

['super hamburguer', 'cordero patagonico']

parte b

In [60]:
recetas_faciles=recetasCordero.filter(lambda x:(x[1][1][1]=='facil'))
recetas_faciles.collect()

[(1, ((3, 1), ('super hamburguer', 'facil'))),
 (3, ((3, 2), ('cordero patagonico', 'facil')))]

In [61]:
recetas_faciles=recetas_faciles.map(lambda x:(x[0],x[1][1][0]))
recetas_faciles.collect()

[(1, 'super hamburguer'), (3, 'cordero patagonico')]

In [63]:
ingredientes_por_RecetaRDD.map(lambda x:(x[0],(x[1],x[2]))).collect()

[(1, (3, 1)),
 (1, (10, 3)),
 (1, (11, 1)),
 (1, (12, 1)),
 (2, (2, 1)),
 (2, (8, 5)),
 (2, (9, 1)),
 (2, (10, 1)),
 (3, (3, 2)),
 (3, (4, 1)),
 (3, (11, 1)),
 (4, (10, 1)),
 (4, (11, 1)),
 (4, (12, 1)),
 (5, (5, 2)),
 (5, (4, 1)),
 (6, (6, 1)),
 (6, (8, 2)),
 (6, (4, 3))]

In [64]:
recetas_faciles=recetas_faciles.join(ingredientes_por_RecetaRDD.map(lambda x:(x[0],(x[1],x[2]))))
recetas_faciles.collect()

[(1, ('super hamburguer', (3, 1))),
 (1, ('super hamburguer', (10, 3))),
 (1, ('super hamburguer', (11, 1))),
 (1, ('super hamburguer', (12, 1))),
 (3, ('cordero patagonico', (3, 2))),
 (3, ('cordero patagonico', (4, 1))),
 (3, ('cordero patagonico', (11, 1)))]

In [66]:
recetas_faciles.map(lambda x:(x[1][1][0],x[1][1][1])).join(ingredientesRDD).collect()

[(10, (3, 'queso')),
 (3, (1, 'cordero')),
 (3, (2, 'cordero')),
 (11, (1, 'tomate')),
 (11, (1, 'tomate')),
 (12, (1, 'lechuga')),
 (4, (1, 'papa'))]

In [67]:
recetas_faciles=recetas_faciles.map(lambda x:(x[1][1][0],x[1][1][1]))
recetas_faciles.join(ingredientesRDD).map(lambda x:(x[1][1],x[1][0]))\
                .reduceByKey(lambda x,y:x+y).collect()

[('cordero', 3), ('tomate', 2), ('lechuga', 1), ('papa', 1), ('queso', 3)]

In [6]:
ingredientesRDD.filter(lambda x:x[1]=="cordero").collect()

[(3, 'cordero')]

In [7]:
ingredientesconCordero=ingredientesRDD.filter(lambda x:x[1]=="cordero").cache()

In [8]:
ingredientesconCordero.collect()

[(3, 'cordero')]

In [9]:
ingredientesRDD.map(lambda x:(x[1],x[0])).collect()

[('hamburguesa', 1),
 ('ravioles', 2),
 ('cordero', 3),
 ('papa', 4),
 ('salmon', 5),
 ('asado', 6),
 ('aceite', 7),
 ('sal', 8),
 ('salsa roja', 9),
 ('queso', 10),
 ('tomate', 11),
 ('lechuga', 12)]

In [10]:
ingredientes_por_RecetaRDD.map(lambda x:(x[1],x[0])).join(ingredientesconCordero).collect()

[(3, (1, 'cordero')), (3, (3, 'cordero'))]

In [11]:
recetas_con_Cordero=ingredientes_por_RecetaRDD.map(lambda x:(x[1],x[0])).join(ingredientesconCordero)\
                            .map(lambda x:(x[1][0],x[1][1]))\
                            .join(recetasRDD).map(lambda x:(x[0],x[1][1]))
recetas_con_Cordero.collect()

[(3, 'cordero patagonico'), (1, 'super hamburguer')]

# cantidad total de cada ingrediente de las recetas con cordero

In [12]:
recetasRDD.filter(lambda x: x[3]=="facil").join(recetas_con_Cordero).collect()

[(1, ('super hamburguer', 'super hamburguer')),
 (3, ('cordero patagonico', 'cordero patagonico'))]

In [13]:
recetas_faciles_con_cordero=recetasRDD.filter(lambda x: x[3]=="facil").join(recetas_con_Cordero).cache()
recetas_faciles_con_cordero.collect()       

[(1, ('super hamburguer', 'super hamburguer')),
 (3, ('cordero patagonico', 'cordero patagonico'))]

In [14]:
ingredientes_por_RecetaRDD.map(lambda x:(x[0],(x[1],x[2]))).join(recetas_faciles_con_cordero) \
                          .collect()

[(1, ((3, 1), ('super hamburguer', 'super hamburguer'))),
 (1, ((10, 3), ('super hamburguer', 'super hamburguer'))),
 (1, ((11, 1), ('super hamburguer', 'super hamburguer'))),
 (1, ((12, 1), ('super hamburguer', 'super hamburguer'))),
 (3, ((3, 2), ('cordero patagonico', 'cordero patagonico'))),
 (3, ((4, 1), ('cordero patagonico', 'cordero patagonico'))),
 (3, ((11, 1), ('cordero patagonico', 'cordero patagonico')))]

In [15]:
ingredientes_por_RecetaRDD.map(lambda x:(x[0],(x[1],x[2]))).join(recetas_faciles_con_cordero) \
                          .map(lambda x:(x[1][0][0],x[1][0][1])).collect()

[(3, 1), (10, 3), (11, 1), (12, 1), (3, 2), (4, 1), (11, 1)]

In [16]:
ingredientes_por_RecetaRDD.map(lambda x:(x[0],(x[1],x[2]))).join(recetas_faciles_con_cordero) \
                          .map(lambda x:(x[1][0][0],x[1][0][1]))\
                          .reduceByKey(lambda x,y: x+y) \
                          .collect() 

[(10, 3), (11, 2), (12, 1), (3, 3), (4, 1)]

In [17]:
ingredientes_por_RecetaRDD.map(lambda x:(x[0],(x[1],x[2]))).join(recetas_faciles_con_cordero) \
                          .map(lambda x:(x[1][0][0],x[1][0][1]))\
                          .reduceByKey(lambda x,y: x+y) \
                          .join(ingredientesRDD) \
                          .map(lambda x:(x[1][1],x[1][0]))\
                          .collect()  

[('lechuga', 1), ('cordero', 3), ('queso', 3), ('papa', 1), ('tomate', 2)]

# Ejercicio de Spark 2do cuatrimestre,2do recuperatorio 2018

In [18]:
# usuarios=(id_usuario,pais, nickname, fecha_alta)
#publicaciones=(id_publicacion, id_usuario, id_foto, comentario, fecha)

In [4]:
usuarios=[[1,"Argentina", "capo123", "20160304"],
         [2,"Afganistan", "life12", "20170304"],
         [3,"Brasil", "panDeDios", "20160304"],
         [4,"Cuba", "fidel", "20160304"],
         [5,"Argentina", "cheChe", "20180304"],
        [6,"Brasil", "LulaPreso", "20190304"],
         [7,"Argentina", "nenem2020", "20160304"]]

publicaciones=[[1,2,3,"este es un comentario #estaloco", "20170304"],
              [2,1,3,"comentario #estaEnamorado", "20170304"],
              [3,3,3,"no se que #pasa", "20170304"],
              [4,4,3,"que linda foro #lindafoto", "20170304"],
              [5,7,3," no entiedo #estaloco a este tipo ", "20170304"],
              [6,7,3,"este es un #cafelatte comentario #camáñapresidencial", "20170304"],
              [7,6,3,"este es un comentario #eshermosa", "20170304"],
              [8,5,3,"este comentario es #feo", "20170304"],
              [9,5,3,"este comentario es #feo", "20170304"]]


In [5]:
usuariosRDD=sc.parallelize(usuarios,4)
publicacionesRDD=sc.parallelize(publicaciones,4)
usuariosRDD.collect()
publicacionesRDD.collect()

[[1, 2, 3, 'este es un comentario #estaloco', '20170304'],
 [2, 1, 3, 'comentario #estaEnamorado', '20170304'],
 [3, 3, 3, 'no se que #pasa', '20170304'],
 [4, 4, 3, 'que linda foro #lindafoto', '20170304'],
 [5, 7, 3, ' no entiedo #estaloco a este tipo ', '20170304'],
 [6, 7, 3, 'este es un #cafelatte comentario #camáñapresidencial', '20170304'],
 [7, 6, 3, 'este es un comentario #eshermosa', '20170304'],
 [8, 5, 3, 'este comentario es #feo', '20170304'],
 [9, 5, 3, 'este comentario es #feo', '20170304']]

# Top 10 hastags publicados por usuarios de argentina

Primero hay que armar registros del tipo (id_usuario,comenatrios). Para eso hay hacer un flatmap y filtrar si las palabras tienen un #. obtenemos un rdd del tipo (id_usuario, #hastag)
Luego tenemos que filtar los usuarios que no son de argentina y hacemos un inner join entre este rdd (id_usuario, pais) con el antererior. obtenemos un rdd (#, id_usario) de todos los hashtags que no queremos. 
Con el ultimo rdd hacemos un right join con el rdd (#, id usario). Todos los registros que tengan None en el termino x[1][0], son los hashtags que queremos

In [9]:
def tienehashtag(word):
    if '#' in word :return True
    else:
        return False

In [10]:
tienehashtag("hola")

False

In [11]:
tienehashtag("#hola")

True

In [12]:
hashpublicados=publicacionesRDD.flatMap(lambda x:[(x[1],word) for word in x[3].split()])\
               .filter(lambda x:tienehashtag(x[1])).cache()
hashpublicados.collect()

[(2, '#estaloco'),
 (1, '#estaEnamorado'),
 (3, '#pasa'),
 (4, '#lindafoto'),
 (7, '#estaloco'),
 (7, '#cafelatte'),
 (7, '#camáñapresidencial'),
 (6, '#eshermosa'),
 (5, '#feo'),
 (5, '#feo')]

In [13]:
usuariosNoArgentinos=usuariosRDD.filter(lambda x :x[1]!="Argentina").map(lambda x:(x[0],x[1])).cache()
usuariosNoArgentinos.collect()

[(2, 'Afganistan'), (3, 'Brasil'), (4, 'Cuba'), (6, 'Brasil')]

In [26]:
usuariosNoArgentinos.join(hashpublicados).collect()

[(2, ('Afganistan', '#estaloco')),
 (3, ('Brasil', '#pasa')),
 (4, ('Cuba', '#lindafoto')),
 (6, ('Brasil', '#eshermosa'))]

In [27]:
hashNoArgentinos=usuariosNoArgentinos.join(hashpublicados).map(lambda x:(x[1][1],x[0])).distinct().cache()
hashNoArgentinos.collect()

[('#eshermosa', 6), ('#estaloco', 2), ('#lindafoto', 4), ('#pasa', 3)]

In [28]:
hashNoArgentinos.rightOuterJoin(hashpublicados.map(lambda x:(x[1],x[0]))).collect()

[('#eshermosa', (6, 6)),
 ('#pasa', (3, 3)),
 ('#cafelatte', (None, 7)),
 ('#feo', (None, 5)),
 ('#feo', (None, 5)),
 ('#lindafoto', (4, 4)),
 ('#estaEnamorado', (None, 1)),
 ('#estaloco', (2, 2)),
 ('#estaloco', (2, 7)),
 ('#camáñapresidencial', (None, 7))]

In [29]:
hashArhentinos=hashNoArgentinos.rightOuterJoin(hashpublicados.map(lambda x:(x[1],x[0])))\
                .filter(lambda x: x[1][0]==None).map(lambda x:(x[0],x[1][1])).cache()
hashArhentinos.collect()
                

[('#cafelatte', 7),
 ('#feo', 5),
 ('#feo', 5),
 ('#estaEnamorado', 1),
 ('#camáñapresidencial', 7)]

In [30]:
hashArhentinos.join(hashpublicados.map(lambda x:(x[1],x[0]))).collect() 

[('#cafelatte', (7, 7)),
 ('#feo', (5, 5)),
 ('#feo', (5, 5)),
 ('#feo', (5, 5)),
 ('#feo', (5, 5)),
 ('#camáñapresidencial', (7, 7)),
 ('#estaEnamorado', (1, 1))]

In [31]:
hashArhentinos.join(hashpublicados.map(lambda x:(x[1],x[0])))\
              .map(lambda x:(x[0],1)).reduceByKey(lambda x,y:x+y)\
              .takeOrdered(10, key=lambda x: -x[1])          

[('#feo', 4),
 ('#cafelatte', 1),
 ('#camáñapresidencial', 1),
 ('#estaEnamorado', 1)]

# Construir RDD pares de id usuario que tienen hashtags en comun

In [14]:
hashRDD=hashpublicados.map(lambda x:(x[1],x[0])).cache()
hashRDD.collect()

[('#estaloco', 2),
 ('#estaEnamorado', 1),
 ('#pasa', 3),
 ('#lindafoto', 4),
 ('#estaloco', 7),
 ('#cafelatte', 7),
 ('#camáñapresidencial', 7),
 ('#eshermosa', 6),
 ('#feo', 5),
 ('#feo', 5)]

In [15]:
hashRDD.join(hashRDD).collect()

[('#estaloco', (2, 2)),
 ('#estaloco', (2, 7)),
 ('#estaloco', (7, 2)),
 ('#estaloco', (7, 7)),
 ('#eshermosa', (6, 6)),
 ('#lindafoto', (4, 4)),
 ('#cafelatte', (7, 7)),
 ('#pasa', (3, 3)),
 ('#camáñapresidencial', (7, 7)),
 ('#feo', (5, 5)),
 ('#feo', (5, 5)),
 ('#feo', (5, 5)),
 ('#feo', (5, 5)),
 ('#estaEnamorado', (1, 1))]

In [17]:
hashRDD.join(hashRDD).distinct().filter(lambda x:x[1][0]!=x[1][1]).collect()

[('#estaloco', (2, 7)), ('#estaloco', (7, 2))]

In [19]:
lista=hashRDD.join(hashRDD).distinct().filter(lambda x:x[1][0]!=x[1][1]).map(lambda x: list(x[1])).collect()

In [20]:
lista

[[2, 7], [7, 2]]

In [21]:
publicacionesRDD.flatMap(lambda x: [(x[1],(x[0],line)) for line in  x[3].split()])\
                .filter(lambda x:hayUnhastag(x[1][1]))\
                .take(3)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 15.0 failed 1 times, most recent failure: Lost task 0.0 in stage 15.0 (TID 91, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/jorge/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/home/jorge/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/jorge/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 390, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/jorge/anaconda3/lib/python3.7/site-packages/pyspark/rdd.py", line 1354, in takeUpToNumLeft
   

In [ ]:
publicacionesRDD.flatMap(lambda x: [(x[1],(x[0],line)) for line in  x[3].split()])\
                .filter(lambda x:hayUnhastag(x[1][1]))\
                .map(lambda x:(x[1][1],x[0])).collect()

In [ ]:
hashtag_id=publicacionesRDD.flatMap(lambda x: [(x[1],(x[0],line)) for line in  x[3].split()])\
                .filter(lambda x:hayUnhastag(x[1][1]))\
                .map(lambda x:(x[1][1],x[0])).cache()

In [ ]:
hashtag_id.join(hashtag_id).collect()

In [ ]:
hashtag_id.join(hashtag_id).filter(lambda x:x[1][0]!=x[1][1]).distinct().collect()

Ejercicio de pandas de 2 parcial de 2018. Spotify!

In [ ]:
canciones=[['040519',1,3,"La cucaracha","Los Redondos"],
            ['040519',2,4,"symphony for love","Metallica"],
             ['030519',3,3,"La cucaracha","Los Redondos"],
            ['030519',4,5,"Rock del gato","Macri y su  pandilla"],
             ['040519',5,3,"La cucaracha","Los Redondos"],
          ['030519',6,6,"Symfonia numero 8","Beethoven"],
          ['040519',7,3,"La cucaracha","Los Redondos"],
          ['040519',8,3,"La cucaracha","Los Redondos"],
          ['040519',9,5,"Rock del gato","Macri y su  pandilla"]]

tags=[[3,"rock pesado"],
      [4,"pop"],
      [5,"rock pesado"],
      [6,"clasica"],
      [7,"rising"],
      [9,"rock pesado"]]

In [ ]:
cancionesRDD=sc.parallelize(canciones)
tagsRDD=sc.parallelize(tags)

In [ ]:
cancionesRDD.collect()

In [ ]:
tagsRDD.collect()

In [ ]:
from datetime import date
from datetime import datetime, timedelta
date.today()

In [ ]:

yesterday = date.today() - timedelta(1)
print (yesterday.strftime('%d%m%y'))

In [ ]:
hoy=date.today().strftime('%d%m%y')
ayer=(date.today() - timedelta(1)).strftime('%d%m%y')

In [ ]:
hoy

In [ ]:
#primero filtro  rl rdd canciones por fecha de ayer y hoy por separado
# con el formato (id_song, fecha, 1)
cancionesRDD.map(lambda x:(x[2],x[0])).filter(lambda x:x[1]==ayer).collect()

In [ ]:
canciones_hoy=cancionesRDD.map(lambda x:(x[2],x[0])).filter(lambda x:x[1]==hoy).cache()
canciones_ayer=cancionesRDD.map(lambda x:(x[2],x[0])).filter(lambda x:x[1]==ayer).cache()

In [ ]:
canciones_hoy.map(lambda x:(x[0],1)).reduceByKey(lambda a,b:a+b).collect()


In [ ]:
resumen_hoy=canciones_hoy.map(lambda x:(x[0],1)).reduceByKey(lambda a,b:a+b).cache()
resumen_ayer=canciones_ayer.map(lambda x:(x[0],1)).reduceByKey(lambda a,b:a+b).cache()
resumen_ayer.take(10)

In [ ]:
resumen_hoy.leftOuterJoin(resumen_ayer).collect()

In [ ]:
def reemplazarNone(x):
    if x[1]==None: return (x[0],0)
    else:
        return x
        
    

In [ ]:
resumen_hoy.leftOuterJoin(resumen_ayer).mapValues(reemplazarNone).collect()

In [ ]:
rising=resumen_hoy.leftOuterJoin(resumen_ayer).mapValues(reemplazarNone)\
          .filter(lambda x:x[1][0]>x[1][1])\
          .map(lambda x:(x[0],'rising')).cache()
rising.collect()

In [ ]:
#union sirve para hacer un append
tagsRDD.union(rising).collect()

# Ejercicio instagram pandas
(id_usuario,nickname, pais, fecha_alta , cantidad_seguidores, cant_seguidos, total_publicaciones)

In [23]:
users=[[1,"koo","Argentina","10122019",10,20,100],
      [2,"Jorge","Brasil","10122019",39,22,13],
      [3,"John33","EEUU","10122019",100,30,1000],
      [4,"Ale12","Argentina","10122019",32,20,100],
      [5,"Fer19","Brasil","10122019",22,15,120],
      [6,"koo2","Argentina","10122019",39,10,100],
      [7,"LaLula","Brasil","10122019",35,50,100],
      [8,"Obama2000","EEUU","10122019",110,130,69],
      [9,"BrasiCampeon","Brasil","10122019",25,20,100],
      [10,"koo44","Argentina","10122019",37,20,100],
      [11,"koo89","Argentina","10122019",333,20,100]]

In [24]:
import pandas as pd
import numpy as np
usersDF=pd.DataFrame(users,columns = ['id_user','nick','country',
                     'date','num_followers','num_followed','total_publications'])
usersDF

,id_user,nick,country,date,num_followers,num_followed,total_publications
0,1,koo,Argentina,10122019,10,20,100
1,2,Jorge,Brasil,10122019,39,22,13
2,3,John33,EEUU,10122019,100,30,1000
3,4,Ale12,Argentina,10122019,32,20,100
4,5,Fer19,Brasil,10122019,22,15,120
5,6,koo2,Argentina,10122019,39,10,100
6,7,LaLula,Brasil,10122019,35,50,100
7,8,Obama2000,EEUU,10122019,110,130,69
8,9,BrasiCampeon,Brasil,10122019,25,20,100
9,10,koo44,Argentina,10122019,37,20,100


In [26]:
usersDF['Max_seguidores']=usersDF.groupby('country')['num_followers'].transform(lambda x: max(x))
usersDF

,id_user,nick,country,date,num_followers,num_followed,total_publications,Max_seguidores
0,1,koo,Argentina,10122019,10,20,100,333
1,2,Jorge,Brasil,10122019,39,22,13,39
2,3,John33,EEUU,10122019,100,30,1000,110
3,4,Ale12,Argentina,10122019,32,20,100,333
4,5,Fer19,Brasil,10122019,22,15,120,39
5,6,koo2,Argentina,10122019,39,10,100,333
6,7,LaLula,Brasil,10122019,35,50,100,39
7,8,Obama2000,EEUU,10122019,110,130,69,110
8,9,BrasiCampeon,Brasil,10122019,25,20,100,39
9,10,koo44,Argentina,10122019,37,20,100,333


In [28]:
influencers=usersDF[(usersDF['num_followers']>0.8 * usersDF['Max_seguidores'])&(usersDF['num_followed']<1.15*usersDF['num_followers'])]
influencers

,id_user,nick,country,date,num_followers,num_followed,total_publications,Max_seguidores
1,2,Jorge,Brasil,10122019,39,22,13,39
2,3,John33,EEUU,10122019,100,30,1000,110
10,11,koo89,Argentina,10122019,333,20,100,333


In [42]:
#caclulo el maximo de seguidores por pais usando transform
usersDF['maximo_seguidores']=usersDF.groupby('country')['num_followers'].transform('max')
usersDF

,id_user,nick,country,date,num_followers,num_followed,total_publications,maximo_seguidores
0,1,koo,Argentina,10122019,10,20,100,333
1,2,Jorge,Brasil,10122019,39,22,13,39
2,3,John33,EEUU,10122019,100,30,1000,110
3,4,Ale12,Argentina,10122019,32,20,100,333
4,5,Fer19,Brasil,10122019,22,15,120,39
5,6,koo2,Argentina,10122019,39,10,100,333
6,7,LaLula,Brasil,10122019,35,50,100,39
7,8,Obama2000,EEUU,10122019,110,130,69,110
8,9,BrasiCampeon,Brasil,10122019,25,20,100,39
9,10,koo44,Argentina,10122019,37,20,100,333


In [43]:
usersDF.groupby('country')['num_followed'].sum().nlargest(1)

country
EEUU    160
Name: num_followed, dtype: int64

In [79]:
users2=usersDF[usersDF['num_followers']>=0.8*usersDF['maximo_seguidores']]
users2

,id_user,nick,country,date,num_followers,num_followed,total_publications,maximo_seguidores
1,2,Jorge,Brasil,10122019,39,22,13,39
2,3,John33,EEUU,10122019,100,30,1000,110
6,7,LaLula,Brasil,10122019,35,50,100,39
7,8,Obama2000,EEUU,10122019,110,130,69,110
10,11,koo89,Argentina,10122019,333,20,100,333


In [81]:
users2[users2['num_followed']<1.15*users2['num_followers']]

,id_user,nick,country,date,num_followers,num_followed,total_publications,maximo_seguidores
1,2,Jorge,Brasil,10122019,39,22,13,39
2,3,John33,EEUU,10122019,100,30,1000,110
10,11,koo89,Argentina,10122019,333,20,100,333


#Ejercio de spark primer parcial 2019
El servicio meteorologico registra datos del tiempo para todas las ciudades donde posea una base de medicion. La informacion se encuentra en dos RDD
Bases_RDD(Id_base, nombre, provincia, ciudad, latitud, long)
mediciones_RDD(timestamp, id_base, temperatura, humedad, presion, direccion_viento, velocidad_viento)
Se pide: reporte para las bases de la provincia buenos aires  que contenga id y nombre de las bases que hayan regoistrado una variación de temperatura promedio del 30 % con respecto al mes anteriro para 2018.

In [31]:
bases=[[1,"capitalRules","CABA","Ciudad de Buenos Aires",37,50],
      [2,"saltita","Salta","Tartagal",37,50],
       [3,"laPlataFolows","Buenos Aires","La Plata",37,50],
       [4,"laMisionera","Misiones","Puerto Argentino",37,50],
       [5,"laformoseña","Formosa","Formosa",37,50],
       [6,"josecito","Buenos Aires","Jose C Paz",37,50],
       [7,"santacruceña","Santa Cruz", "Rio Gallegos",37,50],
       [8,"santita","Santa Fe","Santa Fe",37,50],
       [9,"saltooo","Buenos Aires","Salto",37,50],
       [10,"lacordobeza","Cordoba","Tanti",37,50],
       [11,"laBallena","Buenos Aires","Bahia Blanca",37,50]]

mediciones=[['2018-01-01 12:00:00',3,30,80, 1012, "NO",30],
           ['2018-01-01 12:00:00',4,33,80, 1012, "NO",30],
           ['2018-01-01 12:00:00',5,34,80, 1012, "NO",30],
           ['2018-01-01 12:00:00',6,31,80, 1012, "NO",30],
           ['2018-01-01 12:00:00',1,30,80, 1012, "NO",30],
           ['2018-01-01 12:00:00',2,22,80, 1012, "NO",30],
           ['2019-02-01 12:00:00',3,28,80, 1012, "NO",30],
           ['2018-02-01 12:00:00',10,26,80, 1012, "NO",30],
           ['2018-02-01 12:00:00',9,32,80, 1012, "NO",30],
           ['2018-02-01 12:00:00',2,34,80, 1012, "NO",30],
           ['2018-02-01 12:00:00',3,31,80, 1012, "NO",30],
           ['2017-02-01 12:00:00',3,33,80, 1012, "NO",30],
           ['2018-02-01 12:00:00',3,15,80, 1012, "NO",30],
           ['2018-02-01 12:00:00',3,20,80, 1012, "NO",30],
           ['2018-02-01 12:00:00',9,22,80, 1012, "NO",30],
           ['2018-02-01 12:00:00',3,32,80, 1012, "NO",30],
            ['2018-03-01 12:00:00',3,30,80, 1012, "NO",30],
           ['2018-03-01 12:00:00',4,33,80, 1012, "NO",30],
           ['2018-03-01 12:00:00',5,34,80, 1012, "NO",30],
           ['2018-03-01 12:00:00',6,31,80, 1012, "NO",30],
           ['2018-03-01 12:00:00',1,30,80, 1012, "NO",30],
           ['2018-03-01 12:00:00',2,22,80, 1012, "NO",30],
           ['2019-03-01 12:00:00',3,38,80, 1012, "NO",30],
           ['2018-04-01 12:00:00',10,26,80, 1012, "NO",30],
           ['2018-04-01 12:00:00',9,32,80, 1012, "NO",30],
           ['2018-04-01 12:00:00',2,34,80, 1012, "NO",30],
           ['2018-04-01 12:00:00',3,30,80, 1012, "NO",30],
           ['2017-04-01 12:00:00',3,33,80, 1012, "NO",30],
           ['2018-05-01 12:00:00',3,30,80, 1012, "NO",30],
           ['2018-05-01 12:00:00',3,39,80, 1012, "NO",30],
           ['2018-05-01 12:00:00',9,22,80, 1012, "NO",30],
           ['2018-05-01 12:00:00',3,31,80, 1012, "NO",30],
           ]
     

In [32]:
basesRDD=sc.parallelize(bases)
medicionesRDD=sc.parallelize(mediciones)


In [33]:
basesRDD.take(5)

[[1, 'capitalRules', 'CABA', 'Ciudad de Buenos Aires', 37, 50],
 [2, 'saltita', 'Salta', 'Tartagal', 37, 50],
 [3, 'laPlataFolows', 'Buenos Aires', 'La Plata', 37, 50],
 [4, 'laMisionera', 'Misiones', 'Puerto Argentino', 37, 50],
 [5, 'laformoseña', 'Formosa', 'Formosa', 37, 50]]

In [34]:
medicionesRDD.take(5)

[['2018-01-01 12:00:00', 3, 30, 80, 1012, 'NO', 30],
 ['2018-01-01 12:00:00', 4, 33, 80, 1012, 'NO', 30],
 ['2018-01-01 12:00:00', 5, 34, 80, 1012, 'NO', 30],
 ['2018-01-01 12:00:00', 6, 31, 80, 1012, 'NO', 30],
 ['2018-01-01 12:00:00', 1, 30, 80, 1012, 'NO', 30]]

In [36]:
#filtro basese buenos aires y obtengo registros del tipo (id_base, nombre)
basesBuenosAires=basesRDD.filter(lambda x: x[2]=="Buenos Aires").map(lambda x:(x[0],x[1]))
basesBuenosAires.collect()

[(3, 'laPlataFolows'), (6, 'josecito'), (9, 'saltooo'), (11, 'laBallena')]

In [37]:
def MONTH(timestamp):
    month = ''
    guardar=0
    for caracter in timestamp:
        if (caracter=='-') & (guardar==1):
            return month   
        if (caracter=='-') & (guardar==0):
            guardar=1
        if  (caracter!='-') & (guardar==1):    
            month+=caracter
    return month

In [38]:
def YEAR(timestamp):
    year = ''
    guardar=0
    for caracter in timestamp:
        if (caracter=='-'):
            return year   
        year+=caracter

In [42]:
mediciones2018=medicionesRDD.filter(lambda x:(x[0]>='2017-12-01 00:00:00')& (x[0]<'2019-01-01 00:00:00') )\
                            .map(lambda x:(x[1],(YEAR(x[0]),MONTH(x[0]),x[2])))

mediciones2018.collect()                         

[(3, ('2018', '01', 30)),
 (4, ('2018', '01', 33)),
 (5, ('2018', '01', 34)),
 (6, ('2018', '01', 31)),
 (1, ('2018', '01', 30)),
 (2, ('2018', '01', 22)),
 (10, ('2018', '02', 26)),
 (9, ('2018', '02', 32)),
 (2, ('2018', '02', 34)),
 (3, ('2018', '02', 31)),
 (3, ('2018', '02', 15)),
 (3, ('2018', '02', 20)),
 (9, ('2018', '02', 22)),
 (3, ('2018', '02', 32)),
 (3, ('2018', '03', 30)),
 (4, ('2018', '03', 33)),
 (5, ('2018', '03', 34)),
 (6, ('2018', '03', 31)),
 (1, ('2018', '03', 30)),
 (2, ('2018', '03', 22)),
 (10, ('2018', '04', 26)),
 (9, ('2018', '04', 32)),
 (2, ('2018', '04', 34)),
 (3, ('2018', '04', 30)),
 (3, ('2018', '05', 30)),
 (3, ('2018', '05', 39)),
 (9, ('2018', '05', 22)),
 (3, ('2018', '05', 31))]

In [44]:
bases_joinned=basesBuenosAires.join(mediciones2018)
bases_joinned.collect()

[(9, ('saltooo', ('2018', '02', 32))),
 (9, ('saltooo', ('2018', '02', 22))),
 (9, ('saltooo', ('2018', '04', 32))),
 (9, ('saltooo', ('2018', '05', 22))),
 (6, ('josecito', ('2018', '01', 31))),
 (6, ('josecito', ('2018', '03', 31))),
 (3, ('laPlataFolows', ('2018', '01', 30))),
 (3, ('laPlataFolows', ('2018', '02', 31))),
 (3, ('laPlataFolows', ('2018', '02', 15))),
 (3, ('laPlataFolows', ('2018', '02', 20))),
 (3, ('laPlataFolows', ('2018', '02', 32))),
 (3, ('laPlataFolows', ('2018', '03', 30))),
 (3, ('laPlataFolows', ('2018', '04', 30))),
 (3, ('laPlataFolows', ('2018', '05', 30))),
 (3, ('laPlataFolows', ('2018', '05', 39))),
 (3, ('laPlataFolows', ('2018', '05', 31)))]

In [ ]:
bases_joinned.map(lambda x:(x[0],(x[1][0],x[1][1][0],x[1][1][1],))

In [356]:
bases_buenosAires=basesRDD.filter(lambda x: x[2]=='Buenos Aires')
bases_buenosAires.take(10)

[[3, 'laPlataFolows', 'Buenos Aires', 'La Plata', 37, 50],
 [6, 'josecito', 'Buenos Aires', 'Jose C Paz', 37, 50],
 [9, 'saltooo', 'Buenos Aires', 'Salto', 37, 50],
 [11, 'laBallena', 'Buenos Aires', 'Bahia Blanca', 37, 50]]

In [357]:
mediciones2018=medicionesRDD.filter(lambda x: ((x[0]>='2017-12-01') & (x[0]<'2019-01-01')))\
                            .map(lambda x:(x[1],(x[0],x[2])))
mediciones2018.take(3)

[(3, ('2018-01-01 12:00:00', 30)),
 (4, ('2018-01-01 12:00:00', 33)),
 (5, ('2018-01-01 12:00:00', 34))]

In [358]:
def MONTH(timestamp):
    year = ''
    guardar=0
    for caracter in timestamp:
        if (caracter=='-') & (guardar==1):
            return year   
        if (caracter=='-') & (guardar==0):
            guardar=1
        if  (caracter!='-') & (guardar==1):    
            year+=caracter
    return year
       

In [359]:
MONTH('2018-01-01 12:00:00')

'01'

In [360]:
def YEAR(timestamp):
    year = ''
    guardar=0
    for caracter in timestamp:
        if (caracter=='-'):
            return year   
        year+=caracter

In [361]:
YEAR('2018-01-01 12:00:00')

'2018'

In [362]:
bases_join=bases_buenosAires.join(mediciones2018)
bases_join.take(3)

[(9, ('saltooo', ('2018-02-01 12:00:00', 32))),
 (9, ('saltooo', ('2018-02-01 12:00:00', 22))),
 (9, ('saltooo', ('2018-04-01 12:00:00', 32)))]

In [363]:
med=bases_join.map(lambda x:((x[0],YEAR(x[1][1][0]),MONTH(x[1][1][0]),x[1][0]),(x[1][1][1],1)))
med.take(10)

[((9, '2018', '02', 'saltooo'), (32, 1)),
 ((9, '2018', '02', 'saltooo'), (22, 1)),
 ((9, '2018', '04', 'saltooo'), (32, 1)),
 ((9, '2018', '05', 'saltooo'), (22, 1)),
 ((6, '2018', '01', 'josecito'), (31, 1)),
 ((6, '2018', '03', 'josecito'), (31, 1)),
 ((3, '2018', '01', 'laPlataFolows'), (30, 1)),
 ((3, '2018', '02', 'laPlataFolows'), (31, 1)),
 ((3, '2018', '02', 'laPlataFolows'), (15, 1)),
 ((3, '2018', '02', 'laPlataFolows'), (20, 1))]

In [364]:
med2=med.reduceByKey(lambda x, y: (x[0]+y[0] ,x[1]+y[1]))
med2.take(3)

[((3, '2018', '02', 'laPlataFolows'), (98, 4)),
 ((3, '2018', '04', 'laPlataFolows'), (30, 1)),
 ((9, '2018', '05', 'saltooo'), (22, 1))]

In [367]:
med3=med2.map(lambda x: (x[0][0],(x[0][1],x[0][2],x[0][3],x[1][0]/x[1][1])))
med3.take(16)

[(3, ('2018', '02', 'laPlataFolows', 24.5)),
 (3, ('2018', '04', 'laPlataFolows', 30.0)),
 (9, ('2018', '05', 'saltooo', 22.0)),
 (9, ('2018', '02', 'saltooo', 27.0)),
 (9, ('2018', '04', 'saltooo', 32.0)),
 (6, ('2018', '01', 'josecito', 31.0)),
 (3, ('2018', '01', 'laPlataFolows', 30.0)),
 (6, ('2018', '03', 'josecito', 31.0)),
 (3, ('2018', '03', 'laPlataFolows', 30.0)),
 (3, ('2018', '05', 'laPlataFolows', 33.333333333333336))]

In [371]:
med3.groupByKey().map(lambda x : (x[0], list(x[1]))).collect()

[(9,
  [('2018', '05', 'saltooo', 22.0),
   ('2018', '02', 'saltooo', 27.0),
   ('2018', '04', 'saltooo', 32.0)]),
 (6, [('2018', '01', 'josecito', 31.0), ('2018', '03', 'josecito', 31.0)]),
 (3,
  [('2018', '02', 'laPlataFolows', 24.5),
   ('2018', '04', 'laPlataFolows', 30.0),
   ('2018', '01', 'laPlataFolows', 30.0),
   ('2018', '03', 'laPlataFolows', 30.0),
   ('2018', '05', 'laPlataFolows', 33.333333333333336)])]

In [372]:
int('05')

5

In [48]:
import operator
def compara(lista):
    mes=0
    tem=0
    lista.sort(key = operator.itemgetter(0, 1)) # ordeno por año y mes
    #sorted(lista, key = lambda x: (x[0], x[1]))
    for tupla in lista:
            if (lista.index(tupla)==0):
                mes=int(tupla[1])
                tem=int(tupla[3])
            else: 
                if (int(tupla[1])==mes +1):
                    if ((int(tupla[3]))>=1.3*tem) |((int(tupla[3]))<=tem/1.3):
                        return tupla[2] # devuleve el no,bvre
                mes=int(tupla[1])
                tem=int(tupla[3])
    
    return 'Ninguna'



In [49]:
lista=   [('2018', '05', 'saltooo', 22.0),
   ('2018', '02', 'saltooo', 27.0),
   ('2018', '04', 'saltooo', 32.0)]

In [50]:
compara(lista)

'saltooo'

In [389]:
respuesta=med3.groupByKey().map(lambda x : (x[0], list(x[1]))).map(lambda x: (x[0],compara(x[1])))\
              
respuesta.take(30)

[(9, 'saltooo'), (6, 'Ninguna'), (3, 'Ninguna')]

In [390]:
respuesta=med3.groupByKey().map(lambda x : (x[0], list(x[1]))).map(lambda x: (x[0],compara(x[1])))\
                            .filter(lambda x: x[1]!='Ninguna')
              
respuesta.take(30)

[(9, 'saltooo')]

In [393]:
respuesta=med3.groupByKey().map(lambda x : (x[0], list(x[1]))).map(lambda x: (x[0],x[1][0][2]))
                            
              
respuesta.take(30)

[(9, 'saltooo'), (6, 'josecito'), (3, 'laPlataFolows')]

# Parcial 1 c 2014
1) Se tiene un archivo con información sobre visitas a páginas
web de la forma: (URL, visitas, fecha). Existe solo un registro
por día para cada URL. Se quiere generar un archivo que, por
cada URL, indique cuál fue la fecha en la que tuvo mas visitas
y la cantidad de visitas.
Programar lo pedido en Map Reduce usando agregación para
minimizar la cantidad de datos que deben transferirse en la
red.

In [29]:
visitas=[["google.com",37,'2018-02-01'],
      ["google.com",100,'2018-03-01'],
        ["google.com",1000,'2018-04-01'],
        ["yahoo.com",317,'2018-02-01'],
        ["lanacion.com",52,'2018-02-01'],
        ["spotify.com",183,'2018-02-01'],
        ["clarin.com",274,'2018-02-01'],
        ["lanacion.com",337,'2018-03-01'],
        ["google.com",37,'2018-05-01'],
         ["google.com",57,'2018-06-01'],
        ["clarin.com",300,'2018-03-01'],
        ["lanacion.com",17,'2018-04-01'],
         ["yahoo.com",67,'2018-03-01'],
        ["google.com",31,'2018-07-01'],
        ["spotify.com",19,'2018-02-01'],
         ["yahoo.com",4000,'2018-04-01'],
         ["google.com",3472,'2018-08-01'],
         ["spotify.com",337,'2018-04-01'],
         ["lanacion.com",17,'2018-06-01']]

In [30]:
visitas_rdd=sc.parallelize(visitas)
visitas_rdd.take(3)

[['google.com', 37, '2018-02-01'],
 ['google.com', 100, '2018-03-01'],
 ['google.com', 1000, '2018-04-01']]

In [31]:
respuesta_rdd=visitas_rdd.map(lambda x:(x[0],(x[2],x[1])))\
                .reduceByKey(lambda x,y: (x[0] if x[1] >= y[1] else y[0],max(x[1],y[1])))\
                .map(lambda x:(x[0],x[1][0],x[1][1]))

In [32]:
respuesta_rdd.collect()

[('google.com', '2018-08-01', 3472),
 ('yahoo.com', '2018-04-01', 4000),
 ('lanacion.com', '2018-03-01', 337),
 ('spotify.com', '2018-04-01', 337),
 ('clarin.com', '2018-03-01', 300)]

In [36]:
#ottra forma
respuesta_rdd=visitas_rdd.map(lambda x:(x[0],(x[2],x[1])))\
                .reduceByKey(lambda x,y: (x[0] if x[1] >= y[1] else y[0],x[1] if x[1] >= y[1] else y[1]))\
                .map(lambda x:(x[0],x[1][0],x[1][1]))

In [37]:
respuesta_rdd.collect()

[('google.com', '2018-08-01', 3472),
 ('yahoo.com', '2018-04-01', 4000),
 ('lanacion.com', '2018-03-01', 337),
 ('spotify.com', '2018-04-01', 337),
 ('clarin.com', '2018-03-01', 300)]

 # Parcial 1 c 2014 2da oportunidad
 ) Se tiene un archivo distribuido con información de una red
social en la cual cada registro tiene información sobre un
usuario y una lista de sus amigos y sus enemigos (user_id,
vector_ids_amigos,vector_ids_enemigos). Queremos encontrar
al usuario que figura en mas listas de enemigos, para
designarlo el enemigo público número uno (***) (8 pts por
programar lo pedido) (5 pts por programar la agregació

 

In [18]:
AmigosEnemigos=[[0,[1,2,4],[1,2,4]],
                [1,[0,2,3],[0,6,1]],
                [2,[1,2,4],[0,4,3]],
                [3,[1,2,4],[0,5,6]],
                [4,[5,2,4],[1,2,3]],
                [5,[6,2,4],[1,0,7]],
                [6,[5,2,4],[1,2,4]]]

                
                

In [19]:
AmigosEnemigos

[[0, [1, 2, 4], [1, 2, 4]],
 [1, [0, 2, 3], [0, 6, 1]],
 [2, [1, 2, 4], [0, 4, 3]],
 [3, [1, 2, 4], [0, 5, 6]],
 [4, [5, 2, 4], [1, 2, 3]],
 [5, [6, 2, 4], [1, 0, 7]],
 [6, [5, 2, 4], [1, 2, 4]]]

In [20]:
enemigos_RDD=sc.parallelize(AmigosEnemigos)

In [21]:
enemigos_RDD.take(2)

[[0, [1, 2, 4], [1, 2, 4]], [1, [0, 2, 3], [0, 6, 1]]]

In [23]:
enemigos_RDD.flatMap(lambda x:[(id,1) for id in x[2]]).take()

[(1, 1),
 (2, 1),
 (4, 1),
 (0, 1),
 (6, 1),
 (1, 1),
 (0, 1),
 (4, 1),
 (3, 1),
 (0, 1)]

In [25]:
enemigos_RDD.flatMap(lambda x:[(id,1) for id in x[2]])\
            .reduceByKey(lambda x,y: x+y)\
            .reduce(lambda x,y: x if x[1]>y[1] else y )

(1, 5)